In [2]:
import pickle
import os

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.signal import resample

from datetime import datetime

### Merge datasets 1_mod + 2_mod:

In [3]:
sales = pd.read_csv("E:/Capstone-Retail/data_dsmarket/new_files/item_sales_mod.csv") #Marina Windows
#sales = pd.read_csv("/Users/marinaramiropareta/Desktop/Capstone_Retail/data_dsmarket/new_files/item_sales_mod.csv") #Marina Mac

del(sales["Unnamed: 0"])
sales.head()

,id,item,category,department,store,store_code,region,d,units
0,ACCESORIES_1_001_NYC_1,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,d_1,0
1,ACCESORIES_1_002_NYC_1,ACCESORIES_1_002,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,d_1,0
2,ACCESORIES_1_003_NYC_1,ACCESORIES_1_003,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,d_1,0
3,ACCESORIES_1_004_NYC_1,ACCESORIES_1_004,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,d_1,0
4,ACCESORIES_1_005_NYC_1,ACCESORIES_1_005,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,d_1,0


In [4]:
calendar = pd.read_csv("E:/Capstone-Retail/data_dsmarket/new_files/daily_calendar_with_events_mod.csv") #Marina Windows
#calendar = pd.read_csv("/Users/marinaramiropareta/Desktop/Capstone_Retail/data_dsmarket/new_files/daily_calendar_with_events_mod.csv") #Marina Mac

del(calendar["Unnamed: 0"])
calendar.head()

,date,weekday,weekday_int,d,event,week,year,year_week
0,2011-01-29,Saturday,1,d_1,0,4,2011,2011-4
1,2011-01-30,Sunday,2,d_2,0,4,2011,2011-4
2,2011-01-31,Monday,3,d_3,0,5,2011,2011-5
3,2011-02-01,Tuesday,4,d_4,0,5,2011,2011-5
4,2011-02-02,Wednesday,5,d_5,0,5,2011,2011-5


In [5]:
## Merge datasets 1_mod + 2_mod:
df = pd.merge(
    left = sales,
    right = calendar,
    how = 'left',
    on = ["d"]
    )
# Archivo similar a: "sales_calendar.csv"
df

,id,item,category,department,store,store_code,region,d,units,date,weekday,weekday_int,event,week,year,year_week
0,ACCESORIES_1_001_NYC_1,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,d_1,0,2011-01-29,Saturday,1,0,4,2011,2011-4
1,ACCESORIES_1_002_NYC_1,ACCESORIES_1_002,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,d_1,0,2011-01-29,Saturday,1,0,4,2011,2011-4
2,ACCESORIES_1_003_NYC_1,ACCESORIES_1_003,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,d_1,0,2011-01-29,Saturday,1,0,4,2011,2011-4
3,ACCESORIES_1_004_NYC_1,ACCESORIES_1_004,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,d_1,0,2011-01-29,Saturday,1,0,4,2011,2011-4
4,ACCESORIES_1_005_NYC_1,ACCESORIES_1_005,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,d_1,0,2011-01-29,Saturday,1,0,4,2011,2011-4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58327365,SUPERMARKET_3_823_PHI_3,SUPERMARKET_3_823,SUPERMARKET,SUPERMARKET_3,Queen_Village,PHI_3,Philadelphia,d_1913,1,2016-04-24,Sunday,2,0,16,2016,2016-16
58327366,SUPERMARKET_3_824_PHI_3,SUPERMARKET_3_824,SUPERMARKET,SUPERMARKET_3,Queen_Village,PHI_3,Philadelphia,d_1913,0,2016-04-24,Sunday,2,0,16,2016,2016-16
58327367,SUPERMARKET_3_825_PHI_3,SUPERMARKET_3_825,SUPERMARKET,SUPERMARKET_3,Queen_Village,PHI_3,Philadelphia,d_1913,0,2016-04-24,Sunday,2,0,16,2016,2016-16
58327368,SUPERMARKET_3_826_PHI_3,SUPERMARKET_3_826,SUPERMARKET,SUPERMARKET_3,Queen_Village,PHI_3,Philadelphia,d_1913,3,2016-04-24,Sunday,2,0,16,2016,2016-16


## Preprocessing

In [6]:
df.shape

(58327370, 16)

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 58327370 entries, 0 to 58327369
Data columns (total 16 columns):
 #   Column       Dtype 
---  ------       ----- 
 0   id           object
 1   item         object
 2   category     object
 3   department   object
 4   store        object
 5   store_code   object
 6   region       object
 7   d            object
 8   units        int64 
 9   date         object
 10  weekday      object
 11  weekday_int  int64 
 12  event        object
 13  week         int64 
 14  year         int64 
 15  year_week    object
dtypes: int64(4), object(12)
memory usage: 7.4+ GB


## Feature engineering - Part 2 - Using all data ("days")

### Item clustering

#### Week/Weekend_sales

In [8]:
# Creamos una columna que tenga las ventas de los fines de semana:
df["weekend_sales"] = np.where(df["weekday"].isin(["Saturday", "Sunday"]), df.units, 0)

# Creamos una columna que tenga las ventas durante la semana:
df["week_sales"] = np.where(df["weekday"].isin(["Monday", "Tuesday", "Wednesday", "Thursday", "Friday"]), df.units, 0)

In [9]:
df

,id,item,category,department,store,store_code,region,d,units,date,weekday,weekday_int,event,week,year,year_week,weekend_sales,week_sales
0,ACCESORIES_1_001_NYC_1,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,d_1,0,2011-01-29,Saturday,1,0,4,2011,2011-4,0,0
1,ACCESORIES_1_002_NYC_1,ACCESORIES_1_002,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,d_1,0,2011-01-29,Saturday,1,0,4,2011,2011-4,0,0
2,ACCESORIES_1_003_NYC_1,ACCESORIES_1_003,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,d_1,0,2011-01-29,Saturday,1,0,4,2011,2011-4,0,0
3,ACCESORIES_1_004_NYC_1,ACCESORIES_1_004,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,d_1,0,2011-01-29,Saturday,1,0,4,2011,2011-4,0,0
4,ACCESORIES_1_005_NYC_1,ACCESORIES_1_005,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,d_1,0,2011-01-29,Saturday,1,0,4,2011,2011-4,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58327365,SUPERMARKET_3_823_PHI_3,SUPERMARKET_3_823,SUPERMARKET,SUPERMARKET_3,Queen_Village,PHI_3,Philadelphia,d_1913,1,2016-04-24,Sunday,2,0,16,2016,2016-16,1,0
58327366,SUPERMARKET_3_824_PHI_3,SUPERMARKET_3_824,SUPERMARKET,SUPERMARKET_3,Queen_Village,PHI_3,Philadelphia,d_1913,0,2016-04-24,Sunday,2,0,16,2016,2016-16,0,0
58327367,SUPERMARKET_3_825_PHI_3,SUPERMARKET_3_825,SUPERMARKET,SUPERMARKET_3,Queen_Village,PHI_3,Philadelphia,d_1913,0,2016-04-24,Sunday,2,0,16,2016,2016-16,0,0
58327368,SUPERMARKET_3_826_PHI_3,SUPERMARKET_3_826,SUPERMARKET,SUPERMARKET_3,Queen_Village,PHI_3,Philadelphia,d_1913,3,2016-04-24,Sunday,2,0,16,2016,2016-16,3,0


In [65]:
df_cluster_item_extra = df.groupby(["item"])["week_sales", "weekend_sales"].sum()
df_cluster_item_extra.reset_index(inplace=True)
df_cluster_item_extra

C:\Users\Marina\AppData\Local\Temp\ipykernel_3616\3696242528.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  df_cluster_item_extra = df.groupby(["item"])["week_sales", "weekend_sales"].sum()


,item,week_sales,weekend_sales
0,ACCESORIES_1_001,2731,1362
1,ACCESORIES_1_002,3282,1777
2,ACCESORIES_1_003,891,544
3,ACCESORIES_1_004,25077,14098
4,ACCESORIES_1_005,9371,5250
...,...,...,...
3044,SUPERMARKET_3_823,10025,5363
3045,SUPERMARKET_3_824,5557,2768
3046,SUPERMARKET_3_825,8819,4707
3047,SUPERMARKET_3_826,8443,3745


#### Encontrar las fechas de la primera y última compra para cada artículo:

In [13]:
# Todos los valores
print(len(df))

# Valores con ventas
df_sales = df[df['units'] != 0]
print(len(df_sales))

# Valores sin ventas
df_nans = df[df['units'] == 0]
print(len(df_nans))

print()

58327370
18550276
39777094



In [67]:
#  Se agrupa el DataFrame por la columna "item" y se obtiene la primera fecha con ventas ("df_final_sales")
item_first_last_sale = df_sales.groupby(["item"]).agg({'date': [np.min, np.max]})
item_first_last_sale = item_first_last_sale.reset_index()

# Resetear el nombre de las columnas a un rango de valores numericos
item_first_last_sale.columns = range(item_first_last_sale.shape[1])
# Crear diccionario para mapeo de nombres de columna
lista_1 = item_first_last_sale.columns.tolist()
lista_2 = ["item", "item_first_sale", "item_last_sale"]
dict_dates = dict(zip(lista_1, lista_2))

item_first_last_sale.rename(columns=dict_dates, inplace=True)
item_first_last_sale

,item,item_first_sale,item_last_sale
0,ACCESORIES_1_001,2013-07-16,2016-04-24
1,ACCESORIES_1_002,2011-03-05,2016-04-24
2,ACCESORIES_1_003,2014-02-07,2016-04-24
3,ACCESORIES_1_004,2011-01-29,2016-04-24
4,ACCESORIES_1_005,2011-05-06,2016-04-24
...,...,...,...
3044,SUPERMARKET_3_823,2011-01-29,2016-04-24
3045,SUPERMARKET_3_824,2011-01-29,2016-04-24
3046,SUPERMARKET_3_825,2011-01-29,2016-04-24
3047,SUPERMARKET_3_826,2013-04-03,2016-04-24


#### Calcular el tiempo de exposición:

In [68]:
item_first_last_sale.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3049 entries, 0 to 3048
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   item             3049 non-null   object
 1   item_first_sale  3049 non-null   object
 2   item_last_sale   3049 non-null   object
dtypes: object(3)
memory usage: 71.6+ KB


In [69]:
item_first_last_sale["item_first_sale"] = pd.to_datetime(item_first_last_sale["item_first_sale"])
item_first_last_sale["item_last_sale"] = pd.to_datetime(item_first_last_sale["item_last_sale"])

In [70]:
item_first_last_sale.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3049 entries, 0 to 3048
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   item             3049 non-null   object        
 1   item_first_sale  3049 non-null   datetime64[ns]
 2   item_last_sale   3049 non-null   datetime64[ns]
dtypes: datetime64[ns](2), object(1)
memory usage: 71.6+ KB


In [71]:
item_first_last_sale['item_exhibition_days'] = (item_first_last_sale['item_last_sale'] - item_first_last_sale['item_first_sale']).dt.days
item_first_last_sale.head()

,item,item_first_sale,item_last_sale,item_exhibition_days
0,ACCESORIES_1_001,2013-07-16,2016-04-24,1013
1,ACCESORIES_1_002,2011-03-05,2016-04-24,1877
2,ACCESORIES_1_003,2014-02-07,2016-04-24,807
3,ACCESORIES_1_004,2011-01-29,2016-04-24,1912
4,ACCESORIES_1_005,2011-05-06,2016-04-24,1815


In [72]:
df_cluster_item_extra = pd.merge(
    left = df_cluster_item_extra,
    right = item_first_last_sale,
    how = 'left',
    on = ["item"])

df_cluster_item_extra.head()

,item,week_sales,weekend_sales,item_first_sale,item_last_sale,item_exhibition_days
0,ACCESORIES_1_001,2731,1362,2013-07-16,2016-04-24,1013
1,ACCESORIES_1_002,3282,1777,2011-03-05,2016-04-24,1877
2,ACCESORIES_1_003,891,544,2014-02-07,2016-04-24,807
3,ACCESORIES_1_004,25077,14098,2011-01-29,2016-04-24,1912
4,ACCESORIES_1_005,9371,5250,2011-05-06,2016-04-24,1815


#### Días de compras al año

In [73]:
# Agrupa por ítem y calcula el número de días únicos con compras para cada ítem
num_days_with_purchases_by_item = df_sales.groupby('item')['date'].nunique() #se trata de un array

# Convierte la serie en un DataFrame
df_days_with_purchases = num_days_with_purchases_by_item.reset_index()
df_days_with_purchases.columns = ['item', 'days_with_purchases']

# Limpia
del(num_days_with_purchases_by_item)

df_days_with_purchases

,item,days_with_purchases
0,ACCESORIES_1_001,973
1,ACCESORIES_1_002,1619
2,ACCESORIES_1_003,597
3,ACCESORIES_1_004,1900
4,ACCESORIES_1_005,1765
...,...,...
3044,SUPERMARKET_3_823,1398
3045,SUPERMARKET_3_824,1465
3046,SUPERMARKET_3_825,1542
3047,SUPERMARKET_3_826,1111


In [74]:
df_cluster_item_extra = pd.merge(
    left = df_cluster_item_extra,
    right = df_days_with_purchases,
    how = 'left',
    on = ["item"])

df_cluster_item_extra.head()

,item,week_sales,weekend_sales,item_first_sale,item_last_sale,item_exhibition_days,days_with_purchases
0,ACCESORIES_1_001,2731,1362,2013-07-16,2016-04-24,1013,973
1,ACCESORIES_1_002,3282,1777,2011-03-05,2016-04-24,1877,1619
2,ACCESORIES_1_003,891,544,2014-02-07,2016-04-24,807,597
3,ACCESORIES_1_004,25077,14098,2011-01-29,2016-04-24,1912,1900
4,ACCESORIES_1_005,9371,5250,2011-05-06,2016-04-24,1815,1765


#### Ratio (días de compras/tiempo de exposición)

In [75]:
df_cluster_item_extra["purchase_to_exhib_ratio"] = df_cluster_item_extra["days_with_purchases"] / df_cluster_item_extra["item_exhibition_days"]
df_cluster_item_extra.head()

,item,week_sales,weekend_sales,item_first_sale,item_last_sale,item_exhibition_days,days_with_purchases,purchase_to_exhib_ratio
0,ACCESORIES_1_001,2731,1362,2013-07-16,2016-04-24,1013,973,0.960513
1,ACCESORIES_1_002,3282,1777,2011-03-05,2016-04-24,1877,1619,0.862547
2,ACCESORIES_1_003,891,544,2014-02-07,2016-04-24,807,597,0.739777
3,ACCESORIES_1_004,25077,14098,2011-01-29,2016-04-24,1912,1900,0.993724
4,ACCESORIES_1_005,9371,5250,2011-05-06,2016-04-24,1815,1765,0.972452


In [79]:
df_cluster_item_extra.to_csv("E:/Capstone-Retail/data_dsmarket/task_2_clustering/df_cluster_item_extra.csv", index=False)  #Marina Windows